# 1.a

In [19]:
def hex_table(table):
    header = "   " + " ".join(f"{c:2X}" for c in range(16))
    lines = [header]
    for r, row in enumerate(table):
        line = f"{r:2X} " + " ".join(f"{v:2}" for v in row)
        lines.append(line)
    return "\n".join(lines)


def compute_lat(sbox):
    NL = [[0 for _ in range(16)] for _ in range(16)] # initialize with 0s

    for a in range(16):          # input mask rows
        for b in range(16):      # output mask cols
            cnt = 0              
            for x in range(16):  # all possible inputs x
                y = sbox[x]      # get y
                ip_in = (a & x).bit_count() & 1 # masked input
                ip_out = (b & y).bit_count() & 1 # masked output
                if (ip_in ^ ip_out) == 0: # xor of masked equates to 0
                    cnt += 1
            NL[a][b] = cnt # number of inputs la is true
    return NL

pi_Sp = "7 3 A 5 4 8 9 2 E 6 C 1 D B 0 F".split()

sbox1 = []
for t in pi_Sp:
    v = int(t, 16)
    sbox1.append(v)
print(f"Sbox = {sbox1}")

NL = compute_lat(sbox1)

print("========= Linear Approximation Table N_L =========")
print(hex_table(NL))

Sbox = [7, 3, 10, 5, 4, 8, 9, 2, 14, 6, 12, 1, 13, 11, 0, 15]
========= Linear Approximation Table N_L =========
    0  1  2  3  4  5  6  7  8  9  A  B  C  D  E  F
 0 16  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8
 1  8 10 10  8  6  8  8  6  6  8  8 14  8 10 10  8
 2  8  8  6 10  6  6  8 12  8  8  6 10  6  6  8  4
 3  8  6  8  6  4 10  8  6 10  8 10  8 10  8  6  4
 4  8  8  6  6  6  6  8  8 10  2  8  8  8  8 10 10
 5  8 10  4 10  8 10  8  6  4  6  8  6  8 10  8  6
 6  8  8  4  8  8  8 12  8 10 10  6 10 10 10  6 10
 7  8  6 10  8 10  8 12 10  8  6 10  8  6 12  8  6
 8  8  8  8  8 10 10  6  6 10  6  6 10  4  8  4  8
 9  8  6  6  8  8  6 10  4  8 10 10  8  4  6 10  8
 A  8  8 10 14  8  8 10  6 10  6  8  8 10  6  8  8
 B  8 10  8 10  6  8  6  8 12 10  8  6  6 12 10  8
 C  8  4  6 10  8 12  6 10  8  8 10 10  8  8 10 10
 D  8 10  8  6 10 12 10  8 10  8  6  8  8  6 12  6
 E  8  4  8  8 10  6  6  6  8  8  4  8 10 10 10  6
 F  8  6 10  8  4 10 10  8  6  8  4  6  6  8  8 10


# 1.b

## input mask $X$

$X_7 \oplus X_8$ refers to the 7th and 8th bits of the input block.
In a 16-bit block, bits 5, 6, 7, 8 correspond to the second S-box ($S_2$).The bits 7 and 8 are the 3rd and 4th bits of this S-box.Therefore, the Round 1 Input Mask is $3$ (0011) applied to $S_2$.

## output mask $U^4$

$U^4_1 \oplus U^4_4 \oplus U^4_{13} \oplus U^4_{16}$ refers to bits 1, 4, 13, and 16 at the input to the 4th round (output of Round 3 Permutation).
Bits 1 and 4 correspond to the first S-box ($S_1$). The mask is $9$ (1001). Bits 13 and 16 correspond to the fourth S-box ($S_4$). The mask is $9$ (1001).This confirms that the approximation attacks $S_1$ and $S_4$ in the last round as 4 bits + 4 bits = 8 subkey bits recovered.

## constructing the trail

We need to connect the Input Mask 3 in $S^1_2$ to the Output Mask 9 in $S^4_1$ and $S^4_4$ using $\pi_P$ and $N_L$.

### round 1:

- Active S-box: $S_2$
- Input Mask: $3$ (from $X_7 \oplus X_8$).
- Selection: We check Row 3 of the LAT. Column 4 has value 4
    - The bias for $3 \to 4$ is $(4 - 8)/16 = -4/16 = {-1/4}$.
- Permutation: The mask $4$ (0100) on $S_2$ corresponds to global bit 6, or the the 2nd bit of $S_2$.
    - $\pi_P(6) = 6$.
    - Global bit 6 is the 2nd bit of $S_2$.
    - Result: Input to Round 2 is mask $4$ on $S_2$.

### round 2:

- Active S-box: $S_2$
- Input Mask: $4$.
- Selection: We check Row 4 of the LAT. We need an output that fans out to $S_1$ and $S_4$ in the next round. Column 9 has value 2.
    - The bias for $4 \to 9$ is $(2 - 8)/16 = -6/16 = {-3/8}$.
- Permutation: The mask $9$ (1001) on $S_2$ corresponds to global bits 5 and 8, or 1st and 4th bits of $S_2$.
    - $\pi_P(5) = 2$
    - Global bit 2 is the 2nd bit of $S_1$
    - $\pi_P(8) = 14$ (which is in $S_4$)
    - Global bit 14 is the 2nd bit of $S_4$
    - Result: Input to Round 3 is mask $4$ on $S_1$ and $S_4$. This corresponds to the second bit of $S_1$ and $S_4$

### round 3: 

- Active S-boxes: $S_1$ and $S_4$.
- Input Mask ($S_1$): $4$.
- Input Mask ($S_4$): $4$.
- Selection: We need outputs that match our target mask on bits 1, 4, 13, and 16 (mask 9 for both). Checking Row 4, Col 9 in the LAT again gives Value 2.
    - $S_1$ Bias ($4 \to 9$): $-3/8$.
    - $S_4$ Bias ($4 \to 9$): $-3/8$.
- Permutation Check:
    - $S_1$ Output 9 (bits 1, 4) $\to \pi_P(1)=1, \pi_P(4)=13$. (Target bits 1, 13).
    - $S_4$ Output 9 (bits 13, 16) $\to \pi_P(13)=4, \pi_P(16)=16$. (Target bits 4, 16).
    - Combined bits: 1, 4, 13, 16. This matches the required target mask.

Total Active S-boxes:
- Round 1: 1 (S2)
- Round 2: 1 (S2)
- Round 3: 2 (S1, S4)
- Total: 4 

## Calculate the Total Bias using Piling-Up Lemma

The Piling-Up Lemma states that for $n$ independent linear approximations, the total bias $\epsilon_{total}$ is:
$$\epsilon_{total} = 2^{n-1} \prod_{i=1}^{n} \epsilon_i$$
Where $n$ is the number of active S-boxes ($n=4$).

The Biases ($\epsilon_i$):
- Round 1 ($S_2$): $-1/4$
- Round 2 ($S_2$): $-3/8$
- Round 3 ($S_1$): $-3/8$
- Round 3 ($S_4$): $-3/8$

Calculation:
$$\epsilon_{total} = 2^{4-1} \times \left(-\frac{1}{4}\right) \times \left(-\frac{3}{8}\right) \times \left(-\frac{3}{8}\right) \times \left(-\frac{3}{8}\right)$$
$$\epsilon_{total} = 8 \times \left(-\frac{1}{4}\right) \times \left(-\frac{27}{512}\right)$$
$$\epsilon_{total} = -2 \times \left(-\frac{27}{512}\right)$$
$$\epsilon_{total} = \frac{54}{512}$$
$$\epsilon_{total} = \frac{27}{256}$$

# 1.c
- $for (L_1, L_2) \leftarrow (0, 0) \text{ to } (F, F)$
    - $Count[L_1, L_2] \leftarrow 0$

- $for \text{each } (x, y) \in \mathcal{T}$
    - $\text{for } (L_1, L_2) \leftarrow (0, 0) \text{ to } (F, F)$ 
    - $\text{do}$
        - $v^4_{\langle 1 \rangle} \leftarrow L_1 \oplus y_{\langle 1 \rangle} $
        - $v^4_{\langle 4 \rangle} \leftarrow L_2 \oplus y_{\langle 4 \rangle} $
        - $u^4_{\langle 1 \rangle} \leftarrow \pi_S^{-1}(v^4_{\langle 1 \rangle})$ 
        - $u^4_{\langle 4 \rangle} \leftarrow \pi_S^{-1}(v^4_{\langle 4 \rangle})$ 
        
        - $z \leftarrow x_7 \oplus x_8 \oplus u^4_{1} \oplus u^4_{4} \oplus u^4_{13} \oplus u^4_{16} $
        
        - $\text{if } z = 0 $
            - $Count[L_1, L_2] \leftarrow Count[L_1, L_2] + 1$

- $max \leftarrow -1$
- $for (L_1, L_2) \leftarrow (0, 0) \text{ to } (F, F)$
    - $\text{bias} \leftarrow |Count[L_1, L_2] - T/2|$ 
    - $\text{if } \text{bias} > max $
        - $max \leftarrow \text{bias} $
        - $maxkey \leftarrow (L_1, L_2)$

- $\text{output } (maxkey)$

# 2.a

In [20]:
def compute_lat(sbox):
    ND = [[0 for _ in range(16)] for _ in range(16)]  # initialize with 0s

    for dx in range(16):
        for x in range(16):
            xp = x ^ dx # compute x'
            y = sbox[x]  # get y
            yp = sbox[xp]  # get y'
            dy = y ^ yp # compute output difference
            ND[dx][dy] += 1 # increment count for this input and output difference
    return ND


pi_Spp = "F 0 B D 1 C 6 E 2 9 8 4 5 A 3 7".split()

sbox2 = []
for t in pi_Spp:
    v = int(t, 16)
    sbox2.append(v)
print(f"Sbox = {sbox2}")

ND = compute_lat(sbox2)

print("========= Difference distribution Table N_D =========")
print(hex_table(ND))

Sbox = [15, 0, 11, 13, 1, 12, 6, 14, 2, 9, 8, 4, 5, 10, 3, 7]
========= Difference distribution Table N_D =========
    0  1  2  3  4  5  6  7  8  9  A  B  C  D  E  F
 0 16  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  2  0  2  0  2  0  0  2  2  2  0  4
 2  0  0  2  0  2  0  2  2  0  0  2  0  0  6  0  0
 3  0  2  4  0  0  0  2  0  0  2  2  2  0  0  0  2
 4  0  0  0  6  0  0  0  2  0  0  0  2  2  2  2  0
 5  0  2  0  2  0  2  0  2  2  0  0  2  2  0  0  2
 6  0  4  0  0  0  0  0  0  0  2  2  0  0  2  6  0
 7  0  4  2  0  0  2  2  2  0  0  2  0  2  0  0  0
 8  0  0  0  2  2  2  2  0  0  6  0  0  0  2  0  0
 9  0  2  2  0  0  2  2  0  0  2  0  2  0  2  0  2
 A  0  0  2  2  6  0  0  2  0  2  0  2  0  0  0  0
 B  0  0  2  0  0  0  2  0  2  0  0  4  2  0  0  4
 C  0  0  0  2  0  2  0  0  2  0  8  0  0  0  2  0
 D  0  0  2  0  0  4  2  0  2  0  0  0  2  0  4  0
 E  0  0  0  0  2  0  0  6  2  2  0  0  2  0  2  0
 F  0  2  0  2  2  2  0  0  4  0  0  0  2  0  0  2


# 2.b

To find the differential trail with a propagation ratio of $81/4096$ using the four specified S-boxes, we must identify four transitions that each have a probability of $\frac{6}{16}$ or $\frac{3}{8}$, as $(\frac{3}{8})^4 = \frac{81}{4096}$

### Round 1:

- Active S-box: $S_1$
- Permutation Logic: bits in $S_1$ are 1, 2, 3, 4. Permutes to:
    - $\pi_P(1) = 1$ (in $S_1$)
    - $\pi_P(2) = 5$ (in $S_2$ our target)
    - $\pi_P(3) = 9$ (in $S_3$)
    - $\pi_P(4) = 13$ (in $S_4$)
    $S_1$ outputs active bit 2 where: $\pi_P(2) = 5$ (in $S_2$, our target for Round 2)
- Required Output Mask: 4 corresponding to bit 2
- DDT Lookup: We look for an input that mapts to ouput 4. Row A, column 4 has a value of 6. Input mask is A.

### Round 2:

- Active S-box: $S_2$
- Input Mask: 8 (1000), the incoming bit 5 is the 1st bit of $S_2$ (bits 5-8).
- Permutation Logic: bits in $S_2$ are 5, 6, 7, 8. Permutes to:
    - $\pi_P(5) = 2$ (in $S_1$ our target)
    - $\pi_P(6) = 6$ (in $S_2$)
    - $\pi_P(7) = 10$ (in $S_3$)
    - $\pi_P(8) = 14$ (in $S_4$ our target)
$S_2$ outputs active bits 5 and 8 where: $\pi_P(5) = 2$ (in $S_1$, our first target) $\pi_P(8) = 14$ (in $S_4$, our second target)
- Required Output Mask: 9 (1001), corresponding to bits 1 and 4 of the S-box (global bits 5 and 8).
- DDT Lookup: We look for input 8 that maps to output 9. Row 8, Column 9 has a value of 6.

### Round 3:

- Active S-boxes: $S_1$ and $S_4$
- Input Mask $S_1$: 4 (0100), incoming bit 2 is the 2nd bit of $S_1$.
- Input Mask ($S_4$): 4 (0100), incoming bit 14 is the 2nd bit of $S_4$.
- Permutation Logic (to find subkey bits):
    - $S_1$ outputs bits 3, 4 $\to$ $\pi_P(3)=9$, $\pi_P(4)=13$.
    - $S_4$ outputs bits 3, 4 (global bits 15, 16) $\to$ $\pi_P(15)=12$, $\pi_P(16)=16$.
    - Active bits entering final round: 9, 12, 13, 16.This activates $S_3$ (bits 9-12) and $S_4$ (bits 13-16) in the final key mixing layer, requiring us to find 8 subkey bits.
- Required Output Mask: 3 (0011), corresponding to bits 3 and 4 for both S-boxes.
- DDT Lookup: We look for input 4 that maps to output 3. Row 4, Column 3 has a value of 6.

### Total Probability:

The probability is the product of the probabilities of the active S-boxes:
- $P(S_1^1) = 6/16$
- $P(S_2^2) = 6/16$
- $P(S_1^3) = 6/16$
- $P(S_4^3) = 6/16$
$$P_{total} = \left(\frac{6}{16}\right)^4 = \frac{81}{4096}$$

# 2.c
- $\text{for } (L_1, L_2) \leftarrow (0, 0) \text{ to } (F, F)$
    - $Count[L_1, L_2] \leftarrow 0$

- $\text{for each } (x, y, x^*, y^*) \in \mathcal{T}$
    - $\text{if } (y_{\langle 1 \rangle} = y^*_{\langle 1 \rangle}) \text{ and } (y_{\langle 3 \rangle} = y^*_{\langle 3 \rangle}) $
        - $\text{for } (L_1, L_2) \leftarrow (0, 0) \text{ to } (F, F) $
            - $v^4_{\langle 2 \rangle} \leftarrow L_1 \oplus y_{\langle 2 \rangle} $
            - $(v^4_{\langle 2 \rangle})^* \leftarrow L_1 \oplus y^*_{\langle 2 \rangle} $
            - $u^4_{\langle 2 \rangle} \leftarrow \pi_S^{-1}(v^4_{\langle 2 \rangle}) $
            - $(u^4_{\langle 2 \rangle})^* \leftarrow \pi_S^{-1}((v^4_{\langle 2 \rangle})^*) $
            - $(u^4_{\langle 2 \rangle})' \leftarrow u^4_{\langle 2 \rangle} \oplus (u^4_{\langle 2 \rangle})^* $

            - $v^4_{\langle 4 \rangle} \leftarrow L_2 \oplus y_{\langle 4 \rangle} $
            - $(v^4_{\langle 4 \rangle})^* \leftarrow L_2 \oplus y^*_{\langle 4 \rangle} $
            - $u^4_{\langle 4 \rangle} \leftarrow \pi_S^{-1}(v^4_{\langle 4 \rangle}) $
            - $(u^4_{\langle 4 \rangle})^* \leftarrow \pi_S^{-1}((v^4_{\langle 4 \rangle})^*) $
            - $(u^4_{\langle 4 \rangle})' \leftarrow u^4_{\langle 4 \rangle} \oplus (u^4_{\langle 4 \rangle})^* $

            - $\text{if } ((u^4_{\langle 2 \rangle})' = 0110) \text{ and } ((u^4_{\langle 4 \rangle})' = 0110) $
                - $Count[L_1, L_2] \leftarrow Count[L_1, L_2] + 1$

- $max \leftarrow -1$
- $\text{for } (L_1, L_2) \leftarrow (0, 0) \text{ to } (F, F)$
    - $\text{if } Count[L_1, L_2] > max $
        - $max \leftarrow Count[L_1, L_2] $
        - $maxkey \leftarrow (L_1, L_2)$

- $\text{output } (maxkey)$